In [5]:
# Importing necessary libraries
import pandas as pd
import numpy as np
from catboost import CatBoostRegressor, Pool
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error, r2_score

ModuleNotFoundError: No module named 'catboost'

In [2]:
schema = {
    # --- strings ------------------------------------------------------------
    "location":      "string",
    "type":          "string",
    "block":         "string",
    "street":        "string",
    "storey_range":  "string",   
    "flat_model":     "string",

    # --- numerics -----------------------------------------------------------
    "area_sqm":      "float64",
    "price":         "float64", 
    "commence_date": "Int64",   
    "Quantity_Index":"Int64",
}

In [3]:
data = pd.read_csv("../data.csv", parse_dates=["date"], dtype=schema, usecols=range(11))
data.dtypes

date             datetime64[ns]
location         string[python]
type             string[python]
block            string[python]
street           string[python]
storey_range     string[python]
area_sqm                float64
flat_model       string[python]
commence_date             Int64
Quality_Index             int64
price                   float64
dtype: object

In [4]:
data.head()

,date,location,type,block,street,storey_range,area_sqm,flat_model,commence_date,Quality_Index,price
0,2000-01-01,Austin,3 ROOM,170,Governors Row,07 TO 09,69.0,D,1986,20,147000.0
1,2000-01-01,Austin,3 ROOM,174,Governors Row,04 TO 06,61.0,D,1986,16,144000.0
2,2000-01-01,Austin,3 ROOM,216,Stonleigh Pl,07 TO 09,73.0,K,<NA>,17,159000.0
3,2000-01-01,Austin,3 ROOM,215,Stonleigh Pl,07 TO 09,73.0,K,1976,9,167000.0
4,2000-01-01,Austin,3 ROOM,218,Stonleigh Pl,07 TO 09,67.0,K,1976,17,163000.0
